In [1]:
import re
import numpy as np
import pandas as pd

import jieba

from gensim import corpora, models, similarities
import gensim

In [2]:
# 训练数据


data_path = "news.csv"
df = pd.read_csv(data_path)
df.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
dataset = df[['content', 'feature']]
dataset = dataset.dropna()

dataset

,content,feature
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""..."
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""..."
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""..."
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j..."
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",..."
...,...,...
89606,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,"{""type"":""冰球"",""site"":""新华社"",""url"":""http://home.x..."
89607,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,"{""type"":""宏观经济"",""site"":""新华社"",""url"":""http://home..."
89608,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x..."
89609,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x..."


In [4]:
# 每个类别对应的样本数
target = dataset['feature'].apply(lambda x:json.loads(x)["type"])

counts = target.value_counts()
counts 

体育      15452
其它      12992
社会       9150
政治法律     8193
文化娱乐     6310
        ...  
刑法          1
信息产业        1
观察          1
荆楚楷模        1
            1
Name: feature, Length: 452, dtype: int64

In [5]:
# 样本数大于 100 的类别数
len(counts[counts > 100])

62

In [6]:
# 设置主题数量为 100
num_topics = 100

In [7]:
# 将文本拆分成单词
corpus = dataset['content'].apply(lambda t: list(jieba.cut(t)))
corpus = corpus.to_list()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.368 seconds.
Prefix dict has been built successfully.


In [34]:
# 停用词
stopwords_path = "chinese_stopwords.txt"
stopwords = []
with open(stopwords_path, 'r') as f:
    for line in f.readlines():
        word = line.split('\n')[0]
        stopwords.append(word)

stopwords

['，',
 '的',
 '。',
 '、',
 '在',
 '了',
 '是',
 '\u3000',
 '“',
 '”',
 '和',
 '年',
 '月',
 '：',
 '也',
 '）',
 '为',
 '（',
 '有',
 '%',
 '日',
 '将',
 '中',
 '-',
 '到',
 '与',
 '对',
 ':',
 '\xa0',
 '上',
 '都',
 '等',
 '不',
 '他',
 '》',
 '《',
 '就',
 '但',
 '我',
 '而',
 '这',
 '会',
 '并',
 '；',
 '被',
 '后',
 '人',
 '从',
 '还',
 '1',
 '3',
 '6',
 '以',
 '新',
 '说',
 '7',
 '2',
 '要',
 '5',
 '？',
 '更',
 '于',
 '个',
 '10',
 '大',
 '时',
 '4',
 '多',
 '/',
 '让',
 '其',
 ')',
 '(',
 '很',
 '及',
 '下',
 '',
 '能',
 '—',
 '或',
 '该',
 '她',
 '比',
 '8',
 '元',
 '12',
 '已',
 '向',
 '做',
 '来',
 '前',
 '由',
 '好',
 '.',
 '称',
 '给',
 '最',
 '11',
 '·',
 '据',
 '着',
 '又',
 '至',
 '9',
 '20',
 '！',
 '[',
 ']',
 '去',
 '可',
 '把',
 '则',
 '',
 '一',
 '地',
 '高',
 '吗',
 '30',
 '所',
 '分',
 '较',
 '内',
 '第',
 '里',
 '占',
 '过',
 '15',
 '曾',
 '"',
 '再',
 '人民日报',
 '新闻网',
 '它',
 '况',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '尔后',
 '反过来',
 '反过来说',
 '反之',
 '非但',
 '非徒',
 '否则',
 '嘎',
 '嘎登',
 '该',
 '赶',
 '个',
 '各',
 '各个',
 '各位',
 '各种',
 '各自',
 '给',
 '根据',
 '跟',
 '故',
 '

In [10]:
# 统计词频
from collections import Counter
word_counts = Counter([word for line in corpus for word in line])

word_counts.most_common(10)

[('，', 1360799),
 ('的', 703820),
 ('\\', 669896),
 ('。', 636522),
 ('n', 614636),
 ('\u3000', 408918),
 ('、', 296644),
 ('在', 264633),
 ('“', 239982),
 ('”', 239981)]

In [35]:
for word, cnt in word_counts.most_common(20):
    if word in stopwords:
        print(word)

，
的
。
　
、
在
“
”
）
（
月
日
和
年
了
—
：


In [12]:
# 文本清洗，删除停用词和标点符号

texts = []
for doc in corpus:
    text = []
    for word in doc:
        if word not in stop_words and word.isprintable(
        ) and word not in "‘’“”，、；。？！《》（）//":
            text.append(word)
    if text:
        texts.append(text)

In [13]:
from gensim import corpora, models, similarities
import gensim

In [14]:
# 词典
dictionary = corpora.Dictionary(texts)

# 将单词列表转换成向量
corpus = [dictionary.doc2bow(text) for text in texts]

# 返回每个 doc 对应的单词及词频
print(corpus[20])

[(0, 4), (10, 1), (22, 3), (42, 2), (83, 1), (91, 1), (101, 1), (182, 2), (191, 1), (192, 1), (245, 1), (255, 1), (270, 1), (292, 1), (296, 1), (297, 3), (301, 1), (375, 1), (414, 1), (419, 1), (451, 2), (464, 1), (488, 2), (532, 1), (603, 2), (607, 4), (662, 2), (717, 2), (718, 2), (747, 2), (795, 1), (798, 1), (805, 3), (841, 2), (862, 2), (865, 1), (928, 1), (993, 1), (1091, 1), (1125, 2), (1167, 1), (1171, 1), (1188, 1), (1242, 1), (1245, 3), (1288, 1), (1344, 1), (1452, 1), (1456, 1), (1457, 1), (1527, 1), (1563, 1), (1675, 1), (1681, 1), (1695, 2), (1729, 1), (1766, 3), (1865, 1), (1876, 1), (1916, 1), (1921, 1), (2005, 3), (2051, 1), (2065, 1), (2085, 1), (2203, 2), (2324, 4), (2341, 1), (2349, 2), (2371, 1), (2393, 5), (2491, 1), (2496, 1), (2509, 1), (2514, 1), (2529, 1), (2553, 1), (2577, 1), (2774, 2), (3002, 1), (3003, 1), (3004, 2), (3005, 1), (3006, 1), (3007, 1), (3008, 2), (3009, 1), (3010, 2), (3011, 1), (3012, 1), (3013, 1), (3014, 1), (3015, 1), (3016, 1), (3017, 1),

In [65]:
# 保存词典
# dictionary.save_as_text("dictionary")

dictionary = corpora.Dictionary.load_from_text("dictionary")

In [16]:
# 创建模型
lda = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
)

In [17]:
# 打印第 i 个主题
lda.print_topic(10, topn=10)

'0.108*"澳大利亚" + 0.057*"巴西" + 0.053*"世界杯" + 0.040*"NBA" + 0.040*"篮球" + 0.036*"U20" + 0.024*"骑士队" + 0.023*"西部" + 0.022*"勇士" + 0.021*"骑士"'

In [18]:
lda.print_topics(num_topics=10, num_words=10)

[(33,
  '0.069*"进口" + 0.066*"养殖" + 0.054*"病毒" + 0.040*"检验" + 0.028*"新西兰" + 0.023*"口岸" + 0.022*"议定书" + 0.021*"检疫" + 0.021*"批次" + 0.018*"疫苗"'),
 (9,
  '0.187*"土耳其" + 0.108*"上合" + 0.067*"欧罗巴" + 0.062*"瓷器" + 0.043*"孔子" + 0.022*"法甲" + 0.021*"伊斯坦布尔" + 0.021*"乌兹别克斯坦" + 0.020*"欢度" + 0.016*"特展"'),
 (13,
  '0.095*"博物馆" + 0.038*"摩苏尔" + 0.031*"自行车" + 0.026*"探访" + 0.019*"遗址" + 0.019*"美食" + 0.019*"花园" + 0.016*"旧" + 0.016*"这座" + 0.016*"古城"'),
 (80,
  '0.071*"种子" + 0.069*"长春" + 0.036*"个人电脑" + 0.034*"跨境" + 0.028*"越南" + 0.023*"智" + 0.020*"比特" + 0.020*"波士顿" + 0.018*"电商" + 0.018*"通关"'),
 (63,
  '0.025*"报道" + 0.025*"美国" + 0.015*"媒体" + 0.015*"表示" + 0.012*"声明" + 0.011*"】" + 0.011*"【" + 0.011*"调查" + 0.010*"可能" + 0.010*"朝鲜"'),
 (32,
  '0.205*"日本" + 0.039*"东京" + 0.029*"图书" + 0.025*"阅读" + 0.023*"第十三届" + 0.020*"出版" + 0.019*"安倍" + 0.017*"书" + 0.017*"代表队" + 0.015*"武术"'),
 (29,
  '0.019*"安全" + 0.018*"信息" + 0.016*"管理" + 0.016*"要求" + 0.015*"相关" + 0.014*"规定" + 0.012*"部门" + 0.009*"进行" + 0.009*"使用" + 0.008*"公开"'),
 (66,


In [19]:
# 单词所属的主题
lda.get_term_topics(dictionary.token2id["甘肃"])

[(99, 0.016399866)]

In [20]:
lda.print_topic(46)

'0.075*"铁路" + 0.047*"新华社" + 0.032*"重庆" + 0.030*"公里" + 0.027*"西安" + 0.025*"玫瑰" + 0.023*"列车" + 0.021*"泰国" + 0.021*"段" + 0.020*"高铁"'

In [21]:
# 保存模型
lda.save("lda.model")

lda = gensim.models.ldamodel.LdaModel.load('lda.model')

In [25]:
# 利用模型
bow = lda.id2word.doc2bow(["甘肃", "浙江", "湖北", "深圳"])

bow

[(435, 1), (437, 1), (2284, 1), (22614, 1)]

In [26]:
# 模型推理
lda.inference([bow])

(array([[0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 1.0099869,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01     , 0.01     , 0.01     , 0.01     , 0.01     ,
         0.01     , 0.01 

In [66]:
# 模型，词典，停用词
lda;
dictionary;
stopwords;

In [67]:
sents = [
    "里皮已经回到了欧洲，但他仍有一肚子话想说。",
    "在接受天空体育采访时，里皮表示——自己选择辞去国足主帅是因为继续合作的条件没有了。",
    "“我在中国度过了将近8年的时间，那是一段快乐的时光。带领国家队重要的不是赢得比赛，而是发展运动，我帮助了中国足球发展。”",
    "“之所以选择辞职是因为继续合作的条件没有了。当我相信不再有热情、欲望和信任这些继续合作的自然条件时，我不喜欢赚太多自己不应得的钱。”",
    "关于未来的计划，里皮表示：“我没有重返俱乐部教练席的渴望，我认为自己永远不会再执教俱乐部了。”",
]

In [68]:
# 将句子拆分成单词
def sents2wordlist(sents):
    sents = [[
        w for w in jieba.cut(line) if w not in stopwords and w.isprintable()
        and w not in "‘’“”，、；。？！《》（）//" and w in dictionary.values()
    ] for line in sents]
    return sents


wordlist = sents2wordlist(sents)
wordlist

[['皮', '已经', '回到', '欧洲', '仍', '一肚子', '话', '想'],
 ['接受',
  '天空',
  '体育',
  '采访',
  '皮',
  '表示',
  '选择',
  '辞去',
  '国',
  '足',
  '主帅',
  '是因为',
  '继续',
  '合作',
  '条件',
  '没有'],
 ['中国',
  '度过',
  '将近',
  '时间',
  '一段',
  '快乐',
  '时光',
  '带领',
  '国家队',
  '重要',
  '不是',
  '赢得',
  '比赛',
  '发展',
  '运动',
  '帮助',
  '中国',
  '足球',
  '发展'],
 ['选择',
  '辞职',
  '是因为',
  '继续',
  '合作',
  '条件',
  '没有',
  '当',
  '相信',
  '不再',
  '热情',
  '欲望',
  '信任',
  '继续',
  '合作',
  '自然',
  '条件',
  '喜欢',
  '钱'],
 ['未来',
  '计划',
  '皮',
  '表示',
  '没有',
  '重返',
  '俱乐部',
  '教练席',
  '渴望',
  '认为',
  '永远',
  '不会',
  '执教',
  '俱乐部']]

In [72]:
# 计算每一句话的主题分布
def wordlist2vec(sents):
    vecs = []
    for sent in sents:
        bow = dictionary.doc2bow(sent)
        vec = lda.inference([bow])
        vecs.append(vec[0])
    return np.concatenate(vecs)


vecs = wordlist2vec(wordlist)
vecs

array([[9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 5.9447126e+00,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e-03, 9.9999998e-03,
        9.9999998e-03, 9.9999998e-03, 9.9999998e

In [74]:
vecs.shape

(5, 100)

In [78]:
# 计算整个段落的主题分布
def doc2vec(sents):
    doc = [w for sent in sents for w in sent]
    bow = dictionary.doc2bow(doc)
    vec = lda.inference([bow])
    return vec[0]

doc_vec = doc2vec(wordlist)
doc_vec.shape

(1, 100)

In [82]:
# 计算每个句子和段落的相似度
ranks = np.matmul(doc_vec, vecs.T)[0]
ranks

array([207.72192, 347.36575, 182.95474, 452.2631 , 183.73203],
      dtype=float32)

In [83]:
ranked_sentences = sorted([(ranks[i], s) for i, s in enumerate(sents)],
                          reverse=True)
ranked_sentences

[(452.2631,
  '“之所以选择辞职是因为继续合作的条件没有了。当我相信不再有热情、欲望和信任这些继续合作的自然条件时，我不喜欢赚太多自己不应得的钱。”'),
 (347.36575, '在接受天空体育采访时，里皮表示——自己选择辞去国足主帅是因为继续合作的条件没有了。'),
 (207.72192, '里皮已经回到了欧洲，但他仍有一肚子话想说。'),
 (183.73203, '关于未来的计划，里皮表示：“我没有重返俱乐部教练席的渴望，我认为自己永远不会再执教俱乐部了。”'),
 (182.95474, '“我在中国度过了将近8年的时间，那是一段快乐的时光。带领国家队重要的不是赢得比赛，而是发展运动，我帮助了中国足球发展。”')]